In [394]:
import pandas as pd
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [395]:
%ls '/gdrive/MyDrive/CE'

 as400idprofile.csv
'BDEX to UDM Data Mapping - May 13, 2021.xlsx'
 bdex_vendv1.csv
 cf.csv
 cntrl.csv
 df1_app_data.csv
 indy_as400.csv
 indy_as400_v1.csv
'Indy Mailed 6.10.2020.xls'
'Indy Mailed 6.10.2020.xlsx'
 invycampv1.csv
 invycntrlv1.csv
 machines.csv
 newvends.csv
'Simpli.fi Indy Campaign June 1 to July 31.xlsx'
'Tables & Rows.gsheet'


In [396]:
mc= pd.read_csv('/gdrive/MyDrive/CE/machines.csv', names=['as400id', 'machine_num'])

In [397]:
 mc.as400id= mc.as400id.map(lambda x: ''.join([i for i in x if i.isdigit()]))

In [398]:
mc.dtypes

as400id        object
machine_num     int64
dtype: object

In [399]:
df= pd.read_csv('/gdrive/MyDrive/CE/as400idprofile.csv', names= ['as400id', 'Region','Industry','IndustrySubType', 'LineOfBiz', 'AvgAge', 'PropertyClass', 'NumLaundryRooms'])

In [400]:
df.as400id= df.as400id.astype(str)

In [401]:
df.shape

(13076, 8)

In [402]:
df= df.merge(mc, how='left', on='as400id')

In [403]:
df.isnull().sum()

as400id               0
Region                0
Industry            162
IndustrySubType     405
LineOfBiz             0
AvgAge             3651
PropertyClass      2939
NumLaundryRooms    2478
machine_num         496
dtype: int64

In [404]:
df.shape

(13076, 9)

In [519]:
df1= pd.read_csv('/gdrive/MyDrive/CE/newvends.csv', low_memory=True, names=['as400id', 'week_num', 'unique_num', 'app_txn', 'first_vends_app', 'non_first_vends_app'])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [522]:
df1.shape

(198311, 6)

In [253]:
df1=df1.loc[df1.week_num <= 16]

In [254]:
df1.sum(axis=0)

week_num                1533944
unique_num              2475174
app_txn                12754381
first_vends_app          219522
non_first_vends_app    12534859
dtype: int64

In [405]:
df2= df1.groupby('as400id')['unique_num', 'app_txn', 'first_vends_app'].sum().reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [406]:
df2['first_vend_pct']= df2.first_vends_app/ df2.app_txn
df2['weekly_avg_unique_usr']= df2.unique_num/16.0
df2['weekly_avg_first_vend']= df2.first_vends_app/16.0
df2['weekly_avg_vends']= df2.app_txn/16.0

In [407]:
df2.head()

,as400id,unique_num,app_txn,first_vends_app,first_vend_pct,weekly_avg_unique_usr,weekly_avg_first_vend,weekly_avg_vends
0,0,565,2364,17,0.007191,35.3125,1.0625,147.7500
1,272,41,157,12,0.076433,2.5625,0.7500,9.8125
2,1039,114,476,3,0.006303,7.1250,0.1875,29.7500
3,1178,70,424,17,0.040094,4.3750,1.0625,26.5000
4,1265,91,616,7,0.011364,5.6875,0.4375,38.5000


In [498]:
#1265: New % in previous 16 weeks
100*7/616

1.1363636363636365

In [500]:
#New
100*(7*(1+2.15/100))/ (616+ (7*(1+2.15/100)))

1.147475609824593

In [501]:
100*(7*(1+2.15/100))/ (616+ (7*(1+2.15/100)))- 100*7/616

0.011111973460956603

In [408]:
df3=df2.merge(df, how='left', on='as400id')

In [413]:
pd.crosstab(df3.PropertyClass, df3.Industry)

Industry,Academic Institutions,Assisted Living,Government,Hospitality Industry,Miscellaneous,Multi-Housing,Recreation/Travel
PropertyClass,,,,,,,
Class A,5,2,0,12,15,119,6
Class A+,0,0,0,0,0,6,0
Class B,3,19,1,50,17,667,46
Class C,9,52,0,96,16,1853,52
Class D,0,5,0,3,1,165,2


In [414]:
bf.as400id.unique()

array([  10824, 1100318, 1100331, ...,  920830,    9279,    9880])

In [415]:
ditch=[]
for a in df3.as400id:
  try:
    int(a)
  except:
    ditch.append(a)

In [416]:
ditch

['7308204-6',
 '75063A',
 '75063a',
 'MIADEMO',
 'NJR2452',
 'NYR9479',
 'ORLDEMO',
 'testDDD']

In [417]:
ditch.append(0)
ditch.append('000000000')

In [418]:
df3=df3.loc[~df3.as400id.isin(ditch)]

In [420]:
df4=df3.loc[df3.PropertyClass.isin(['Class B', 'Class C', 'Class D'])]
# df4=df3

In [425]:
df4.as400id= df4.as400id.astype(int)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [461]:
df4.shape

(3059, 16)

In [427]:
df4.isnull().sum()

as400id                    0
unique_num                 0
app_txn                    0
first_vends_app            0
first_vend_pct             0
weekly_avg_unique_usr      0
weekly_avg_first_vend      0
weekly_avg_vends           0
Region                     0
Industry                   2
IndustrySubType           17
LineOfBiz                  0
AvgAge                   541
PropertyClass              0
NumLaundryRooms          360
machine_num                0
dtype: int64

In [428]:
pwd

'/gdrive'

In [223]:
# df3.as400id.to_csv('as400id.csv', header= False)

# BDEX Data

In [449]:
bd=pd.read_excel('/gdrive/MyDrive/CE/BDEX to UDM Data Mapping - May 13, 2021.xlsx')

In [453]:
bd['udm_user_found']=bd['UDM Consumer User ID'].map({'No Match Found': False}).fillna(True)

In [454]:
bd.udm_user_found.value_counts(normalize=False)*100

False    21953600
True       717600
Name: udm_user_found, dtype: int64

In [455]:
sum(bd['UDM Consumer User ID']== bd['Client Account AS400 ID'])
#


1137

## Client Account AS400 ID:- Address sent to BDEX
### UDM Consumer User ID:- associated as400id with a email through App registration. EDW UDM.Consumer
### things that Mark Did:
 - Address from BDEX address mapped to **Client Account AS400 ID**
 - Email to the udm.Consumer table user_id
 - No Match found-> no user_id
 - No user-id: **TARGET**
 - user_id found:- mapped to **UDM Consumer User ID**

In [456]:
bd.head(1).T

,0
email_address,dennishaymore2@gmail.com
Vendor,DI
Client Account Name,CASA DEL CORONADO
Client Account AS400 ID,3022707
UDM Consumer User ID,337122
UDM Consumer AS400 ID,12
bdex_consumer_id,2B2B797C-5E40-421F-BAE0-FF3CF7FB3987
clean_address,251 W BLACKLIDGE DR
clean_city,TUCSON
clean_state,AZ


In [457]:
met_dict= {'email_address': 'count', 'udm_user_found': 'sum'}

In [458]:
bf= bd.loc[bd.Vendor=='DI'].groupby('Client Account AS400 ID').agg(met_dict).reset_index()
bf.columns=['as400id', 'emails', 'users_found']
bf['opportunity_size']= bf.emails- bf.users_found
bf['bdex']= 'Y'

In [468]:
bf.tail()

,as400id,emails,users_found,opportunity_size,bdex
2210,920182,13,0,13,Y
2211,920187,27,0,27,Y
2212,920830,571,3,568,Y
2213,9279,226,13,213,Y
2214,9880,104,11,93,Y


In [460]:
bf.shape

(2215, 5)

In [462]:
sum(bf.opportunity_size >=10)

1826

In [463]:
bf.as400id= bf.as400id.astype(int)

In [464]:
sum(bf.bdex=='Y')

2215

In [491]:
df5=df4.merge(bf, how='left', on='as400id')
# df4:- as400id vend profile only for Class B,C,D
# bf:- as400ids with target opportunity size

In [492]:
test = df4.merge(bf, how='outer', on='as400id')

In [511]:
test.loc[((test.unique_num.isna()) & (test.bdex=='Y') &(test.opportunity_size >= 10)),'opportunity_size'].sum()

116694.0

In [ ]:
test['venn_flag']= None

In [512]:
test.loc[(test.unique_num >0) & (test.bdex=='Y'), 'venn_flag']= 'Red'
test.loc[(test.unique_num >0) & (test.bdex.isna()), 'venn_flag']= 'Blue'
test.loc[(test.unique_num.isna()) & (test.bdex=='Y'), 'venn_flag']= 'Yellow'

In [514]:
test.venn_flag.value_counts()

Blue      2446
Yellow    1602
Red        613
Name: venn_flag, dtype: int64

In [518]:
test[['as400id', 'venn_flag']].to_csv('venn.csv')

In [496]:
test.as400id.tolist()

[5201388,
 5201408,
 5201410,
 5201662,
 5201732,
 5201746,
 5201768,
 5201776,
 5201814,
 5201819,
 520189,
 520190,
 520195,
 520205,
 520213,
 520218,
 520229,
 520275,
 5202834,
 5203027,
 5203077,
 5203084,
 5203102,
 5203123,
 5203126,
 5203143,
 5203174,
 5203247,
 5203278,
 5203279,
 5203351,
 5203352,
 5203355,
 5203371,
 520345,
 520359,
 520392,
 520400,
 520415,
 520425,
 520437,
 520448,
 520449,
 520453,
 520455,
 520460,
 520461,
 520462,
 520497,
 520500,
 520518,
 520552,
 520560,
 520564,
 520570,
 520576,
 520619,
 520632,
 520737,
 520768,
 520776,
 520779,
 520781,
 520784,
 520799,
 520815,
 520867,
 520878,
 520884,
 520887,
 520925,
 520936,
 520960,
 520976,
 520979,
 521002,
 521008,
 521010,
 521011,
 521031,
 521056,
 521097,
 521110,
 521111,
 521112,
 521118,
 5211336,
 5211343,
 5211362,
 5211368,
 5211396,
 5211398,
 5211418,
 5211467,
 5211476,
 5211515,
 5211524,
 5211537,
 5211554,
 5211578,
 5211583,
 5211601,
 5211605,
 5211609,
 5211617,
 5211620,


In [484]:
len(set(df4.as400id).difference(set(bf.as400id)))

2446

In [481]:
len(set(bf.as400id).difference(set(df4.as400id)))

1602

In [483]:
len(set(bf.as400id).intersection(set(df4.as400id)))

613

In [497]:
1602+613

2215

In [477]:
a= set(df4.as400id)
b= set(bf.as400id) 

In [476]:
a.

AttributeError: ignored

In [466]:
df5.dtypes

as400id                    int64
emails                     int64
users_found                int64
opportunity_size           int64
bdex                      object
unique_num               float64
app_txn                  float64
first_vends_app          float64
first_vend_pct           float64
weekly_avg_unique_usr    float64
weekly_avg_first_vend    float64
weekly_avg_vends         float64
Region                    object
Industry                  object
IndustrySubType           object
LineOfBiz                 object
AvgAge                    object
PropertyClass             object
NumLaundryRooms          float64
machine_num              float64
dtype: object

In [392]:
df5.shape

(2215, 20)

In [393]:
sum(df5.bdex=='Y')

2215

In [321]:
df5.bdex.fillna('N', inplace= True)
df5.opportunity_size.fillna(0, inplace=True)

In [285]:
df5.bdex.value_counts()

N    2780
Y     610
Name: bdex, dtype: int64

In [294]:
df5.columns

Index(['as400id', 'unique_num', 'app_txn', 'first_vends_app', 'first_vend_pct',
       'weekly_avg_unique_usr', 'weekly_avg_first_vend', 'weekly_avg_vends',
       'Region', 'Industry', 'IndustrySubType', 'LineOfBiz', 'AvgAge',
       'PropertyClass', 'NumLaundryRooms', 'machine_num', 'emails',
       'users_found', 'opportunity_size', 'bdex'],
      dtype='object')

In [313]:
overall_first_vent_pct= df5.first_vends_app.sum()/df5.app_txn.sum()
delta_first_vend= 2.15*df5.first_vends_app.sum()/100

In [314]:
delta_first_vend

1257.5135

In [312]:
overall_first_vent_pct

0.015287093332580597

In [305]:
(df5.first_vends_app.sum()+ delta_first_vend)/(df5.app_txn.sum()+ delta_first_vend)

0.01561063505267791

In [315]:
(df5.loc[df5.bdex=='Y'].first_vends_app.sum()+ delta_first_vend)/(df5.loc[df5.bdex=='Y'].app_txn.sum()+ delta_first_vend)

0.015570078109912113

In [316]:
(df5.loc[df5.bdex=='N'].first_vends_app.sum())/(df5.loc[df5.bdex=='N'].app_txn.sum())

0.015618262872192116

In [317]:
(df5.loc[df5.bdex=='Y'].first_vends_app.sum())/(df5.loc[df5.bdex=='Y'].app_txn.sum())

0.013522610181321398

In [318]:
df5.columns

Index(['as400id', 'unique_num', 'app_txn', 'first_vends_app', 'first_vend_pct',
       'weekly_avg_unique_usr', 'weekly_avg_first_vend', 'weekly_avg_vends',
       'Region', 'Industry', 'IndustrySubType', 'LineOfBiz', 'AvgAge',
       'PropertyClass', 'NumLaundryRooms', 'machine_num', 'emails',
       'users_found', 'opportunity_size', 'bdex'],
      dtype='object')

In [319]:
df5.groupby('bdex').agg({'as400id':'count', 'unique_num': 'sum', 'first_vends_app': 'sum', 'app_txn': 'sum'})

,as400id,unique_num,first_vends_app,app_txn
bdex,,,,
N,2780,592270,50313,3221421
Y,610,111397,8176,604617


In [157]:
# Across all as400ids
overall_first_vent_pct= df4.first_vends_app.sum()/ df4.app_txn.sum()
# Current weekly overall first vends
current_weekly_first_vends= df4.first_vends_app.sum()/16.0
# Gross Weekly Increment
gross_weekly_increment_first_vends= (1+ 2.15/100)*current_weekly_first_vends
# Delta required each week
delta_weekly= (2.15/100)*current_weekly_first_vends
# CTR
ctr= .9/100
cvr= 10/100

In [240]:
df4.IndustrySubType.unique()

array([nan, 'Hotel/Motel/Inn', 'Apartments', 'Salon/Spa/Barbershop',
       'Truck Stop', 'Extended Stay', 'Camp/RV Park',
       'College / University', 'Mobile Home/Trailer Park', 'Condominiums',
       'Resorts', 'Housing Authority - Elderly',
       'Senior Community / Private Elderly Hsing',
       'Housing Authority - Subsidized', 'Halfway House/Group Home',
       'Privatized Student Housing', 'Marina/Yact',
       'Camp/Mobile Home/Trailer/RV Park', 'Housing Authority - Family',
       'Marina/Yacht', 'Fraternity/Sorority', 'Store', 'Military',
       'Cooperative', 'YMCA/YWCA', 'Nursing Home', 'Schools'],
      dtype=object)

In [158]:
delta_weekly

294.93565624999997

In [160]:
target.columns

Index(['as400id', 'unique_num', 'app_txn', 'first_vends_app', 'first_vend_pct',
       'weekly_avg_unique_usr', 'weekly_avg_first_vend', 'weekly_avg_vends',
       'Region', 'Industry', 'IndustrySubType', 'LineOfBiz', 'AvgAge',
       'PropertyClass', 'NumLaundryRooms', 'machine_num', 'emails',
       'users_found', 'opportunity_size', 'bdex'],
      dtype='object')

In [239]:
for cutoff in [10,20]:
  slct= ((df4.opportunity_size>= cutoff) & (df4.bdex== 'Y'))
  target= df4.loc[slct]
  others= df4.loc[~slct]
  print(cutoff, len(target), len(others), others.weekly_avg_first_vend.sum()/(others.weekly_avg_vends.sum()), delta_weekly/(target.weekly_avg_vends.sum()+delta_weekly-target.weekly_avg_first_vend.sum()))

10 539 12521 0.017394364610326427 0.008663701190562555
20 442 12618 0.017383552266943332 0.009777048735176744


In [167]:
len(target)

442

In [154]:
target.shape

(442, 20)

In [145]:
current_weekly_first_vends

13717.9375

# Back of the Envelope

In [ ]:
(1+2.15/100)*219522/(2645*16)

5.298717462192817

In [ ]:
219522*1.0215- 219522

4719.723000000027

In [ ]:
len(df1.as400id.unique())

13069

In [ ]:
219522/(13069*16)

1.049822098094728

In [302]:
from statsmodels.stats.power import TTestIndPower

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [303]:
# parameters for power analysis (change as needed)
effect = 0.05
alpha = 0.05
power = 0.8
# perform power analysis #
# change to TTestPower() in case of a paired sample t-test

In [304]:
analysis = TTestIndPower()
result = analysis.solve_power()
print('Sample Size: %.2f' % result)

Sample Size: 6280.05
